In [ ]:
import os
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

labels = os.path.join(os.getenv("SYNTHETIC_LABELS_PATH"), 'labels.csv')

# Load the labels
labels_df = pd.read_csv(labels)

IMG_SIZE = (64, 64)

# Prepare images and labels
images = []
labels = []

for index, row in labels_df.iterrows():
    img_path = os.path.join(os.getenv("SYNTHETIC_IMAGES_DIR"), row['filename'])
    image = load_img(img_path, target_size=IMG_SIZE, color_mode="grayscale") 
    image = img_to_array(image) / 255.0
    images.append(image)
    labels.append(row['label'])

images = np.array(images)
labels = np.array(labels)

# labels_df.info()
print(images, images.shape)
# print(labels, labels.shape)


In [2]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels (convert strings to integer classes)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# One-hot encode the encoded labels
one_hot_labels = to_categorical(encoded_labels)

X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
